# Task 1 of Assignment 2

**Objective:** Train separate models for pathology groups to leverage shared or unique characteristics

**Split Ratio:** 5:3:2 (50% training, 30% validation, 20% testing), applied independently per group

**Pathology Grouping:**

- Lung Diseases: Atelectasis, Consolidation, Edema, Lung Opacity, Pneumonia, Lung Lesion (6)

- Pleural-Related: Effusion, Pleural Other, Pneumothorax (3)

- Cardiac and Mediastinal: Cardiomegaly, Enlarged Cardiomediastinum (2)

- Fracture: Fracture (1)

- Support Devices: Support Devices (1)

**Key Features:**

- Data split with fixed seed (seed=42) for consistency
- NaN values converted to 0, unrecognized pathologies defaulted to 0
- Outputs dictionary with 5 groups, each containing train/validation/test datasets

### 1. Load Data

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from tqdm import tqdm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set TensorFlow logging level to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Helper function: Load embeddings from TFRecord files
def load_embedding(embedding_path):
    raw_dataset = tf.data.TFRecordDataset([embedding_path])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value
    embedding = torch.tensor(embedding_values)
    if torch.isnan(embedding).any() or torch.isinf(embedding).any():
        raise ValueError(f"NaN or Inf found in embedding: {embedding_path}")
    return embedding

# Custom dataset class for MIMIC chest X-ray embeddings with grouped pathologies
class MIMIC_Embed_Dataset(Dataset):
    pathology_groups = {
        "Lung Diseases": ["Atelectasis", "Consolidation", "Edema", "Lung Opacity", "Pneumonia", "Lung Lesion"],
        "Pleural-Related": ["Effusion", "Pleural Other", "Pneumothorax"],
        "Cardiac and Mediastinal": ["Cardiomegaly", "Enlarged Cardiomediastinum"],
        "Fracture": ["Fracture"],
        "Support Devices": ["Support Devices"]
    }

    def __init__(self, embedpath, csvpath, metacsvpath, group_name, views=["PA"], mode="train", seed=42, split_ratio=[0.5, 0.3, 0.2], unique_patients=True):
        super().__init__()
        np.random.seed(seed)
        self.group_name = group_name
        self.pathologies = self.pathology_groups[group_name]
        self.mode = mode
        self.embedpath = embedpath
        self.csv = pd.read_csv(csvpath)
        self.metacsv = pd.read_csv(metacsvpath)
        self.split_ratio = split_ratio

        self.csv = self.csv.set_index(["subject_id", "study_id"])
        self.metacsv = self.metacsv.set_index(["subject_id", "study_id"])
        self.csv = self.csv.join(self.metacsv).reset_index()

        self.csv["view"] = self.csv["ViewPosition"]
        self.limit_to_selected_views(views)

        if unique_patients:
            self.csv = self.csv.groupby("subject_id").first().reset_index()

        n_row = self.csv.shape[0]
        if self.mode == "train":
            self.csv = self.csv[:int(n_row * self.split_ratio[0])]
        elif self.mode == "valid":
            self.csv = self.csv[int(n_row * self.split_ratio[0]):int(n_row * (self.split_ratio[0] + self.split_ratio[1]))]
        elif self.mode == "test":
            self.csv = self.csv[int(n_row * (self.split_ratio[0] + self.split_ratio[1])):]
        else:
            raise ValueError(f"mode must be one of [train, valid, test], got {self.mode}")

        healthy = self.csv["No Finding"] == 1
        labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                self.csv.loc[healthy, pathology] = 0
                mask = self.csv[pathology]
            else:
                mask = pd.Series(0, index=self.csv.index)
            labels.append(mask.values)
        self.labels = np.asarray(labels).T.astype(np.float32)
        self.labels[np.isnan(self.labels)] = 0
        self.labels = np.where((self.labels != 0) & (self.labels != 1), 0, self.labels)
        

        self.csv = self.csv.reset_index(drop=True)
        self.csv["offset_day_int"] = self.csv["StudyDate"]
        self.csv["patientid"] = self.csv["subject_id"].astype(str)

    def limit_to_selected_views(self, views):
        if not isinstance(views, list):
            views = [views]
        if '*' in views:
            views = ["*"]
        self.views = views
        self.csv["view"] = self.csv["view"].fillna("UNKNOWN")
        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {}
        sample["idx"] = idx
        sample["lab"] = self.labels[idx]
        subjectid = str(self.csv.iloc[idx]["subject_id"])
        studyid = str(self.csv.iloc[idx]["study_id"])
        dicom_id = str(self.csv.iloc[idx]["dicom_id"])
        embed_file = os.path.join(self.embedpath, f"p{subjectid[:2]}", f"p{subjectid}", f"s{studyid}", f"{dicom_id}.tfrecord")
        sample["embedding"] = load_embedding(embed_file)
        return sample

# Function to initialize datasets
def initialize_mimic_datasets(embedpath, csvpath, metacsvpath, seed=42, split_ratio=[0.5, 0.3, 0.2]):
    dataset_dict = {}
    group_names = ["Lung Diseases", "Pleural-Related", "Cardiac and Mediastinal", "Fracture", "Support Devices"]
    
    for group in group_names:
        dataset_dict[group] = {
            "train": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="train", seed=seed, split_ratio=split_ratio),
            "valid": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="valid", seed=seed, split_ratio=split_ratio),
            "test": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="test", seed=seed, split_ratio=split_ratio)
        }
    
    return dataset_dict

# Function to print dataset details
def print_dataset_details(datasets, num_samples=3):
    """
    Print details about the datasets to understand their structure and content.
    
    Args:
        datasets (dict): Dictionary of datasets for each pathology group
        num_samples (int): Number of samples to print per dataset
    """
    for group_name, splits in datasets.items():
        print(f"\n=== {group_name} ===")
        for split_name, dataset in splits.items():
            print(f"\n{split_name} Dataset:")
            print(f"Size: {len(dataset)} samples")
            print(f"Pathologies: {dataset.pathologies}")
            print(f"Label shape: {dataset.labels.shape}")
            
            # Print label distribution
            label_means = np.mean(dataset.labels, axis=0)
            print("Label distribution (mean per pathology):")
            for pathology, mean in zip(dataset.pathologies, label_means):
                print(f"  {pathology}: {mean:.3f}")
            
            # Print a few samples
            print(f"\nFirst {min(num_samples, len(dataset))} samples:")
            for i in range(min(num_samples, len(dataset))):
                sample = dataset[i]
                print(f"Sample {i}:")
                print(f"  Index: {sample['idx']}")
                print(f"  Labels: {sample['lab']}")
                print(f"  Embedding shape: {sample['embedding'].shape}")
                print(f"  Embedding (first 5 values): {sample['embedding'][:5]}")
            print("-" * 50)

# Example usage
embedpath = "/Users/maktubcn/Downloads/SPH-Assignment2/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "/Users/maktubcn/Downloads/SPH-Assignment2/data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "/Users/maktubcn/Downloads/SPH-Assignment2/data/mimic-cxr-2.0.0-metadata.csv"

datasets = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

# Print details about the datasets
print_dataset_details(datasets, num_samples=3)



Below is a use case for calling data, the content on the right side of the equal sign can be directly called.

In [ ]:
# Load Lung Diseases
lung_diseases_train = datasets["Lung Diseases"]["train"]
lung_diseases_valid = datasets["Lung Diseases"]["valid"]
lung_diseases_test = datasets["Lung Diseases"]["test"]
# Load Pleural-Related
pleural_related_train = datasets["Pleural-Related"]["train"]
pleural_related_valid = datasets["Pleural-Related"]["valid"]
pleural_related_test = datasets["Pleural-Related"]["test"]
# Load Cardiac and Mediastinal
cardiac_mediastinal_train = datasets["Cardiac and Mediastinal"]["train"]
cardiac_mediastinal_valid = datasets["Cardiac and Mediastinal"]["valid"]
cardiac_mediastinal_test = datasets["Cardiac and Mediastinal"]["test"]  
#Load Fracture
fracture_train = datasets["Fracture"]["train"]
fracture_valid = datasets["Fracture"]["valid"]
fracture_test = datasets["Fracture"]["test"]
#Load Support Devices
support_devices_train = datasets["Support Devices"]["train"]
support_devices_valid = datasets["Support Devices"]["valid"]
support_devices_test = datasets["Support Devices"]["test"]